# Sequential algorithms

The following exercises tests that have understood algorithms and algorithmic design in sufficient detail to reason about termination and correctness and that you have enough experience with Python programming to implement simple sequential (also known as iterative) algorithms.  Here, by sequential, I simply mean that the algorithms iterate through one or more loops to achieve their goal.

For the exercises, we need some code for picking random numbers and for validating user input. That code is listed below. You do not need to understand it to do the exercises, but you do need to evaluate this code before the exercises can run.

In [ ]:
from numpy.random import randint

def input_integer(prompt):
    while True:
        try:
            inp = input(prompt)
            i = int(inp)
            return i
        except:
            print(inp, "is not a valid integer.")

def input_selection(prompt, options):
    modified_prompt = "{} [{}]: ".format(prompt.strip(), ", ".join(options))
    while True:
        inp = input(modified_prompt)
        if inp in options:
            return inp
        else:
            print("Invalid choice! Must be in [{}]".format(", ".join(options)))
        

## Below or above

Here's a game you can play with a friend: one of you think of a number between 1 and 20, both 1 and 20 included. The other has to figure out what that number is. He or she can guess at the number, and after guessing will be told if the guess is right, or if it is too high, or if it is too low. Unless the guess is correct, the guesser must try again until the guess *is* correct.

The following program implements this game for the case where the computer picks the number and you have to guess it. Play with the computer as long as you like.

In [ ]:
# When picking a random mumber, we specify the interval [low,high).
# Since high is not included in the interval, we need to use 1 to 21 to
# get a random number in the interval [1,20].
n = randint(1, 21, size = 1)

# Now, repeat guessing until we get the right number.
guess = input_integer("Make a guess. ")
while guess != n:
    if guess > n:
        print("Your guess is too high!")
    else:
        print("Your guess is too low!")
    guess = input_integer("Make a guess. ")
print("You got it!")

Here are three different strategies you could use to guess the number:
1. Start with 1. If it isn't the right number, it has to be too low--there are no smaller numbers the right one could be. So if it isn't 1, you guess it is 2. If it isn't, you have once again guessed too low, so now you try 3. You continue by incrementing your guess by one until you get the right answer.
2. Alternatively, you start at 20. If the right number is 20, great, you got it in one guess, but if it is not, your guess must be too high--it cannot possibly be too small. So you try 19 instead, and this time you work your way down until you get the right answer.
3. Tired of trying all numbers from one end to the other, you can pick this strategy: you start by guessing 10. If this is correct, you are done, if it is too high, you know the real number must be in the interval $[1,9]$, and if the guess is too low, you know the right answer must be in the interval $[11,20]$--so for your next guess, you pick the middle of the interval it must be. With each new guess, you update the interval where the real number can be hidden and pick the middle of the new interval.

**Exercise:** Prove that all three strategies terminate and with the correct answer, i.e. they are algorithms for solving this problem.

**Exercise:** Would you judge all three approaches to be equally efficient in finding the right number? If not, how would you order the three strategies such that the method most likely to get the right number first is ranked highest and the algorithm most likely to get the right number last is ranked lowest. Justify your answer.

If you do not lie to the computer when it asks you about its guess compared to the number you are thinking of, this program implements the first strategy:

In [ ]:
for guess in range(1,21):
    result = input_selection("How is my guess {}?".format(guess), ["low", "hit", "high"])
    if result == "hit":
        print("Wuhuu!")
        break
    else:
        print("I must have been too low, right? the answer was", result)

**Exercise:** Implement the other two strategies and test them.

When iterating from 20 and down, for the second strategy, you should always get the result `"high"` when you ask about your guess, so you can use a `for` loop and not worry about the actual result form `input_selection`. When you implement strategy number three, however, you need to keep track of a candidate interval with a lower bound, initially 1, and an upper bound, initially 20. When you guess to high, you should lower your upper bound to the value you just guessed minus one (no need to include the guess we know is too high). When you guess to low, you must increase your lower bound to the number you just guessed plus one. In both cases, after updating the interval, you should guess for the middle point in the new interval. When you compute the middle value in your interval, you can use 

```python
guess = int(upper_bound + lower_bound) / 2)
```

## Finding square roots

Given a number positive number $S > 0$, we want to compute its positive square root, $\sqrt{S}$. We don't need our answer to be perfectly accurate. Using floating point numbers with a finite number of bits to represent the uncountable set of real numbers prevents this anyway. However, we want to be able to put an upper bound on the error we get, $\epsilon$, such that we are guaranteed that for our result, $\hat{S}$, we have $|S-\hat{S}|<\epsilon$.

One algorithm that solves this problem is known as the *Babylonian method* and is based on two observations. The first is this, for any $x>0$, if $x>\sqrt{S}$ then $S/x<\sqrt{S}$ and if $S/x>\sqrt{S}$ then $x<\sqrt{S}$, i.e., if we guess at a value for the square root of $S$ and the guess is too high, we have a lower bound on what it *could* be, and if the guess is too low, we have an upper bound on what it could be.

![](figures/Babylonian-method-range.png)

To see this, consider the case where $x>\sqrt{S}$ and therefore $x^2>S$. This inequality naturally also implies that $S/x^2 < x^2/x^2$, and from this we derive $S=S\frac{x^2}{x^2}>S\frac{S}{x^2}=\left(\frac{S}{x^2}\right)^2$, i.e., $S/x<\sqrt{S}$. The other case is proven similarly.

Because of this, if we start out knowing nothing about $\sqrt{S}$, it could be anywhere between $0$ and $S$, so we can make an initial guess of some $x_0$, $0<x_0<S$. If $|S-x|<\epsilon$, then $x_0$ is an acceptable output and we are done. If not, we know that $\sqrt{S}$ lies in the interval $(x/S,x)$ (if $x^2>S$) or in the interval $(x,x/s)$ (if $x^2<S$), and we can make a new guess inside that interval.

The Babylonian method for finding square roots follows this idea and work as follows:
1. First, make a guess for $x_0$, e.g. $x_0=S/2$. Any number in $(0,S)$ will do.
2. Now, repeat the following, where we denote the guess we have at iteration $i$ by $x_i$.

    1. If $|S/x_i-x_i|<\epsilon$ report $x_i$.
    2. Otherwise, update $x_{i+1}=\frac{1}{2}\left(x_i+S/x_i\right)$.
    

The test $|S/x_i-x_i|<\epsilon$ is different than the requirement we made about the error we would accept, which was $|\sqrt{S}-x_i|<\epsilon$, but since we don't know $\sqrt{S}$ we cannot test that directly. We know, however, that $\sqrt{S}$ lies in the interval $(S/x,x)$ or the interval $(x,S/x)$, so if we make this interval smaller than $\epsilon$, we have reached at least the accuracy we want.

The update $x_{i+1}=\frac{1}{2}\left(x_i+S/x_i\right)$ picks the next guess to be the average of $x_i$ and $S/x_i$, which is also the midpoint in the interval $(S/x,x)$ (for $x>S/x$) or the interval $(x,S/x)$ (for $x<S/x$), so inside the interval we know must contain $\sqrt{S}$.

**Exercise:** From this description alone you can argue that *if* the method terminates, it will report a correct answer. Prove that the algorithm is correct.

In each iteration, we update the interval in which we know $\sqrt{S}$ resides by cutting the previous interval in half.

**Exercise:** Use this to prove that the algorithm terminates.

**Exercise:** Implement and test this algorithm.